## Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier


#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

In [4]:
# Determine the number of unique values in each column.
unique_counts = application_df.nunique()

# Display the result
print(unique_counts)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [5]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
print(application_type_counts)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 500
application_types_to_replace = list(application_type_counts[application_type_counts < cutoff_value].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_greater_than_1 = classification_counts[classification_counts > 1]
print(classification_counts_greater_than_1)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64


In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classification_cutoff_value = 1000
classifications_to_replace = list(classification_counts[classification_counts < classification_cutoff_value].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, columns=['APPLICATION_TYPE', 'AFFILIATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', 'CLASSIFICATION'])


In [11]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(columns=['IS_SUCCESSFUL'])
y = application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [18]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()
input_features = len(X_train.columns)

# First hidden layer
nn.add(tf.keras.layers.Dense(units=10, input_dim=input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=5, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 10)                440       
                                                                 
 dense_6 (Dense)             (None, 5)                 55        
                                                                 
 dense_7 (Dense)             (None, 1)                 6         
                                                                 
Total params: 501 (1.96 KB)
Trainable params: 501 (1.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.6030 - accuracy: 0.6964
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5621 - accuracy: 0.7282
Epoch 3/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5571 - accuracy: 0.7288
Epoch 4/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5549 - accuracy: 0.7305
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5536 - accuracy: 0.7311
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5530 - accuracy: 0.7319
Epoch 7/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5521 - accuracy: 0.7314
Epoch 8/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5516 - accuracy: 0.7313
Epoch 9/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5510 - accuracy: 0.7321
Epoch 10/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5503 - accura

804/804 [==============================] - 3s 3ms/step - loss: 0.5425 - accuracy: 0.7374
Epoch 82/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5425 - accuracy: 0.7373
Epoch 83/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5426 - accuracy: 0.7375
Epoch 84/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5423 - accuracy: 0.7374
Epoch 85/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5428 - accuracy: 0.7369
Epoch 86/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5423 - accuracy: 0.7367
Epoch 87/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5422 - accuracy: 0.7373
Epoch 88/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5425 - accuracy: 0.7371
Epoch 89/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5424 - accuracy: 0.7371
Epoch 90/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5423 - accuracy: 

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5532 - accuracy: 0.7243 - 948ms/epoch - 4ms/step
Loss: 0.5532058477401733, Accuracy: 0.7243148684501648


In [24]:
# Initialize the Random Forest model
rf_model = RandomForestClassifier(random_state=42)

# Fit the model to the training data
rf_model.fit(X_train_scaled, y_train)


RandomForestClassifier(random_state=42)

In [29]:
# Evaluate the model on the test data
rf_accuracy = rf_model.score(X_test_scaled, y_test)
print(f"Random Forest Accuracy: {rf_accuracy}")

Random Forest Accuracy: 0.7074052478134111


In [30]:
# Export our model to HDF5 file
nn.save("Results/AlphabetSoupCharity_RandomForest.h5")

C:\Users\jahn_\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
